# Imports

In [3]:
from Assigner import Assigner;
from generate_blendshape import get_blendshapes;
from preprocessing import align_mesh,generate_meshes, get_lm_as_np;
from training_loop import process_video;
import tensorflow as tf;
import os;

# Logical GPU Config for testing multi-processing

In [ ]:
#Generate two different logical GPUs from One GPUs for testing.
gpus = tf.config.list_physical_devices('GPU')
tf.config.set_logical_device_configuration(
      gpus[0],
      [tf.config.LogicalDeviceConfiguration(memory_limit=8000),
        tf.config.LogicalDeviceConfiguration(memory_limit=8000)])
logical_gpus = tf.config.list_logical_devices('GPU')

# Preprocessing

In [5]:
fps, meshes = generate_meshes("./x.mov");
meshes = get_lm_as_np(meshes);

In [6]:
blendshapes = get_blendshapes(meshes, fps);
# Basis Mesh is the first one to be processed.
basis_mesh = blendshapes.pop(0);
basis_mesh, aligned_mesh = align_mesh(basis_mesh, blendshapes);

In [7]:
_, frames_to_process = generate_meshes("./x.mov");
processing_meshes = get_lm_as_np(frames_to_process);
_, frames = align_mesh(basis_mesh, processing_meshes);

# Optimization

In [14]:
# An Example to show multiprocessing works with no data race problems.
file_path_to_save_opt_vals = "./final"
os.mkdir(file_path_to_save_opt_vals);
assigner = Assigner(file_path_to_save_opt_vals, aligned_mesh, frames[:20], basis_mesh, 5500, 0.000095, 10);

# Use the two logical GPUs to test multi gpu processing support.
for i in gpus:
  assigner.assign_frames(i);

#Loop that waits for the threads to join.

while not assigner.check_all_thread_dead():
  for i in list(assigner.active_threads):
    if assigner.active_threads[i] is not None:
      if not assigner.active_threads[i].is_alive():
        pass;
      else:
        assigner.active_threads[i].join();

New Thread Spawned
Frame No: 0 Initial Loss: 0.14475948764341082 Device: /job:localhost/replica:0/task:0/device:CPU:0
Frame No: 0 Final Loss: 0.13766854298783782 Device: /job:localhost/replica:0/task:0/device:CPU:0
Frame No: 8 Initial Loss: 0.06984094723535342 Device: /job:localhost/replica:0/task:0/device:CPU:0
Frame No: 8 Final Loss: 0.06748909088954311 Device: /job:localhost/replica:0/task:0/device:CPU:0
New Thread Spawned
Frame No: 16 Initial Loss: 0.09244527484250895 Device: /job:localhost/replica:0/task:0/device:CPU:0
Frame No: 16 Final Loss: 0.09005623632051725 Device: /job:localhost/replica:0/task:0/device:CPU:0


In [15]:
import threading;
# check number of threads after running multi processing (By default the notebook uses 10 threads)
threading.active_count()

6

In [12]:
dir_to_save_opt_vals = "./final";
os.mkdir(dir_to_save_opt_vals);
process_video(dir_to_save_opt_vals, aligned_mesh, frames, basis_mesh, 5500, 0.025, 2);

Frame No: 0 Initial Loss: 0.15073419219882941 Device: /job:localhost/replica:0/task:0/device:CPU:0
Frame No: 0 Final Loss: 0.010296228999239607 Device: /job:localhost/replica:0/task:0/device:CPU:0
Frame No: 8 Initial Loss: 0.11419437907836563 Device: /job:localhost/replica:0/task:0/device:CPU:0


c:\Program Files\Python37\lib\site-packages\numpy\lib\npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


Frame No: 8 Final Loss: 0.004697859183154741 Device: /job:localhost/replica:0/task:0/device:CPU:0
Frame No: 16 Initial Loss: 0.14503467046108276 Device: /job:localhost/replica:0/task:0/device:CPU:0
Frame No: 16 Final Loss: 0.006449758629392981 Device: /job:localhost/replica:0/task:0/device:CPU:0
Frame No: 24 Initial Loss: 0.08483024501346627 Device: /job:localhost/replica:0/task:0/device:CPU:0
Frame No: 24 Final Loss: 0.005397869479522928 Device: /job:localhost/replica:0/task:0/device:CPU:0
Frame No: 32 Initial Loss: 0.07506715492713029 Device: /job:localhost/replica:0/task:0/device:CPU:0
Frame No: 32 Final Loss: 0.0043041800445389015 Device: /job:localhost/replica:0/task:0/device:CPU:0
Frame No: 40 Initial Loss: 0.12447258697238388 Device: /job:localhost/replica:0/task:0/device:CPU:0
Frame No: 40 Final Loss: 0.005302786408788653 Device: /job:localhost/replica:0/task:0/device:CPU:0
Frame No: 48 Initial Loss: 0.09294290103445835 Device: /job:localhost/replica:0/task:0/device:CPU:0
Frame

KeyboardInterrupt: 